In [1]:
#r "nuget:Microsoft.Spark"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.Spark version 1.0.0

In [2]:
using System;
using System.IO;
using Microsoft.Spark;
using Microsoft.Spark.Sql;
using static Microsoft.Spark.Sql.Functions;

In [11]:
long rows = 1_000_000_000;

In [12]:
// Create a Spark session
SparkSession spark = SparkSession
    .Builder()
    .AppName("montecarlo")
    .GetOrCreate();


In [13]:
DataFrame dataFrame = spark
    .Range(0, rows)
    .Select(Rand().Multiply(2).Minus(1).As("x"), Rand().Multiply(2).Minus(1).As("y"))
    .Select(
        When(
            Col("x").Multiply(Col("x")).Plus(Col("y").Multiply(Col("y"))).Leq(1), 1
        )
        .Otherwise(0)
        .As("InCircle"))
    .Agg(Sum(Col("InCircle")).As("t"))
    .Select(Col("t").Multiply(4).Divide(rows).As("P1"))
    ;

In [14]:
var start = DateTime.Now;

dataFrame.Show();

var stop = DateTime.Now;
Console.WriteLine($"Total Seconds: {(stop-start).TotalSeconds}");

+-----------+
|         P1|
+-----------+
|3.141562896|
+-----------+

Total Seconds: 15,11905


In [13]:
// Stop Spark session
spark.Stop();